# Night Shifts

In [22]:
from src.read_csv import parse_csv
from src.schedule import Person, Schedule

import os
import pandas as pd
import copy
import statistics
from itertools import combinations

In [23]:
# Minimum number of people that need to be assigned during night shifts
min_people_night = 2

In [24]:
availabilities = [parse_csv(os.path.join('data', file)) for file in os.listdir('data') if file.endswith('.csv')]

print(f'Number of availabilities: {len(availabilities)}')
print([avail.columns[0] for avail in availabilities])

Number of availabilities: 7
['UNC', 'Michael Bryant', 'Barack Obama', 'Bill Nye', 'Euler', 'The Rock', 'Blue Devil']


In [25]:
# Sum all the availabilities element-wise, except ignoring if the value is non-numerical
# Therefore, total_avail will contain the total number of people available for each shift

total_avail = None

for df in availabilities:

    df2 = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    if total_avail is None:
        total_avail = df2
    else:
        total_avail += df2

# Add back the times (first column) to the total availability
total_avail = pd.concat([df.iloc[:, 0], total_avail], axis=1)
total_avail.columns = ['Total Available', *total_avail.columns[1:]]

In [26]:
display(total_avail.iloc[-1:, :])

,Total Available,Sunday,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
38,NIGHT SHIFTS,3,3,4,3,5,4,4


In [27]:
people = []

for p in availabilities:
    name = p.columns[0]
    avail_mask = p.iloc[-1:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values[0]
    people.append(Person(name, avail_mask))

night_schedule = Schedule(people, min_people_per_slot=min_people_night)
night_schedule.find_schedules(display)

Schedule balance: 1.1547005383792515


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,"Bill Nye, Euler","Barack Obama, The Rock","Barack Obama, Bill Nye","Euler, The Rock","Michael Bryant, Barack Obama","Euler, Blue Devil","The Rock, Blue Devil"





Schedule balance: 1.1547005383792515


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,"Bill Nye, Euler","Barack Obama, The Rock","Barack Obama, Bill Nye","Euler, The Rock","Michael Bryant, Barack Obama","The Rock, Blue Devil","Euler, Blue Devil"





Schedule balance: 1.1547005383792515


...

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,"Bill Nye, Euler","Barack Obama, The Rock","Barack Obama, Bill Nye","Euler, The Rock","Michael Bryant, Blue Devil","Barack Obama, The Rock","Euler, Blue Devil"





Schedule balance: 1.1547005383792515


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,"Bill Nye, Euler","Barack Obama, The Rock","Barack Obama, Bill Nye","Euler, The Rock","Michael Bryant, Blue Devil","Barack Obama, Blue Devil","Euler, The Rock"





Schedule balance: 1.1547005383792515


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,"Bill Nye, Euler","Barack Obama, The Rock","Barack Obama, Bill Nye","Euler, The Rock","Michael Bryant, Blue Devil","Barack Obama, Blue Devil","Euler, Blue Devil"


In [28]:


# # If balanced_schedules is empty, then there is no possible schedule using strictly people's ideal availabilities
# # Next, we'll randomly keep adding people's 'Q' availabilities until we get a schedule that works, noting which ones we've added
    
# # def unbalanced_schedule():
# #     print("No possible schedule using ideal availabilities\n\n")

# #     class NightScheduleQuestionable(NightSchedule):
# #         # This class is the same as NightSchedule, except it also keeps track of the people who are working during the questionable days
# #         def __init__(self, people, people_working_during_questionable, balanced_schedules):
# #             super().__init__(people)
# #             self.people_working_during_questionable = people_working_during_questionable
# #             self.balanced_schedules = balanced_schedules


# #     # Tabulate all Q availabilities for use
# #     # i.e. if Michael is available [0, 1, 1, 0, Q, Q, Q], then the dictionary will be {'Michael': ['Thursday', 'Friday', 'Saturday']}

# #     unsure_map = {} # All Q availabilities

# #     for avail in availabilities:
# #         name = avail.columns[0]
# #         avail_mask = avail.iloc[-1:, 1:]
# #         # Get indices of all Q's
# #         unsure = avail_mask[avail_mask == 'Q'].dropna(axis=1).columns.tolist()
# #         unsure_map[name] = unsure

# #     unsure_days = [] # ex. ['Thursday', 'Wednesday', 'Saturday', 'Monday', 'Thursday', 'Saturday']
# #     for name in unsure_map:
# #         unsure_days += unsure_map[name]

# #     # Parallel list for who is associated with each day
# #     q_owners = [] # ex. ['Michael', 'John', 'Michael', 'John', 'Michael', 'John']
# #     for name in unsure_map:
# #         q_owners += [name] * len(unsure_map[name])    

# #     total_Q = len(unsure_days)

# #     # Now to keep adding people's Q availabilities until we get a schedule that works
# #     # Start by trying all additions of 1 person's Q availabilities, then 2, etc.
# #     # Break out of the loop once we find a schedule that works


# #     sol_found = False
# #     possible_night_schedules = []

# #     for num_people in range(1, total_Q+1):

# #         # Get all combinations of people's Q availabilities for this number of people

# #         if sol_found:

# #             # Here we have potentionally a bunch of NightScheduleQuestionable objects that work
# #             # Each individually has a parameter balanced_schedules of the top balanced schedules
# #             # We'll now get the top balanced schedules of all of these NightScheduleQuestionable objects, while keeping track of the people_working_during_questionable

# #             # First get all the balances
# #             balances = []
# #             for night_schedule in possible_night_schedules:
# #                 for schedule in night_schedule.balanced_schedules:
# #                     balances.append(schedule.get_balance())

# #             # Establish the cutoff for the top balanced schedules (top 5)
# #             # This will almost certainly produce more than 5 because of ties
# #             cutoff = sorted(balances)[5] if len(balances) > 5 else 99999

# #             # Now just go through all the NightScheduleQuestionable objects and print if the balance is above the cutoff
# #             for night_schedule in possible_night_schedules:
# #                 for schedule in night_schedule.balanced_schedules:
# #                     if schedule.get_balance() <= cutoff:
# #                         print(f"Balance: {schedule.get_balance()} Questionable: {schedule.people_working_during_questionable}")
# #                         print_schedule(schedule)

# #             return

# #         for people in combinations(range(total_Q), num_people):


# #             people_working_during_questionable = {} # To keep track of who we'll be asking to work during the questionable days ex. {'Michael': ['Thursday', 'Saturday'], 'John': ['Wednesday', 'Monday']}
# #             for idx in people:
# #                 name = q_owners[idx]
# #                 day = unsure_days[idx]
# #                 if name not in people_working_during_questionable:
# #                     people_working_during_questionable[name] = []
# #                 people_working_during_questionable[name].append(day)


# #             # As before, now for the people who we are testing to work during the questionable days, we'll expand their availabilities to include the questionable days
# #             people = []
# #             for p in availabilities:
# #                 name = p.columns[0]
# #                 avail_mask = p.iloc[-1:, 1:].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int).values[0]

# #                 if name in people_working_during_questionable:
# #                     # Map 'Sunday' to 0, 'Monday' to 1, etc. from the people_working_during_questionable dictionary
# #                     # Then set the availability mask to 1 for those days
# #                     for day in people_working_during_questionable[name]:
# #                         avail_mask[day_map[day]] = 1

# #                 people.append(NightPerson(name, avail_mask))

# #                 balanced_schedules = [] # The top balanced schedules
# #                 night_schedule = NightScheduleQuestionable(people, people_working_during_questionable, balanced_schedules)
# #                 night_schedule.assign_night_shifts()

# #                 if balanced_schedules:
# #                     possible_night_schedules.append(copy.deepcopy(night_schedule))
# #                     sol_found = True

# #     if not sol_found:
# #         print("No possible schedule using questionable availabilities")

# # if not night_schedule.balanced_schedules:
# #     unbalanced_schedule()
